In [2]:
import time
from datetime import datetime, timedelta
from typing import List, Dict, Optional

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
)

# =========================
# CONFIGURAÇÕES
# =========================
HEADLESS = True        # Se quiser ver o navegador rodando, troque para False
MAX_SCROLLS = 10       # Número máximo de "scrolls" por página
PAUSA_SCROLL = 2       # Pausa (segundos) entre cada scroll
TIMEOUT = 30           # Tempo máximo de espera (segundos) para resultados
DIAS_A_COLETAR = 0     # Quantidade de dias futuros a coletar (0 -> hoje)
SAVE_TO_EXCEL = True   # Salvar em Excel ao final
EXCEL_BASENAME = "passagens_clickbus_rotas"  # prefixo do arquivo
PRINT_ALL_ROWS = True  # mostrar todas as linhas no console

# ROTAS (trechos)
ROTAS: List[Dict[str, str]] = [
    {"origem_url": "vitoria-da-conquista-ba-todos", "destino_url": "sao-paulo-sp-todos"},
    {"origem_url": "sao-paulo-sp-todos", "destino_url": "vitoria-da-conquista-ba-todos"},
]

# =========================
# SELETORES (com alternativas/fallbacks)
# =========================
SEL_DEPARTURE_TIME       = "time.departure-time, time[data-testid='departure-time']"
SEL_COMPANY              = "div.company, div[data-testid='company-name']"
SEL_CLASS                = "div.service-class, span[data-testid='service-class']"
SEL_PRICE                = "span.price-value, span[data-testid='trip-card-price']"
SEL_RESULTS_CONTAINER    = "div.search-results, div[data-testid='search-results']"

# Card de viagem — usando o HTML que você enviou
SEL_TRIP_CARD_CANDIDATES = [
    "div.search-item-main.commodities-enable",      # card externo
    "div[data-testid='search-item-container']"      # inner container
]

# Botões para ABRIR o seatmap
SEL_SELECT_BUTTONS_OPEN = [
    "button[data-testid='select-result-item']",     # botão exato do HTML
    "[data-testid*='select']",
    "[data-testid*='choose']",
    "button[data-testid*='select']",
    "button[data-testid*='choose']",
    "button",
    "a"
]

# Botões para FECHAR (toggle/fechar)
SEL_BUTTONS_CLOSE_WITHIN_CARD = [
    "button[data-testid='select-result-item']",
    "button.select-button",
    "button[data-testid='select-result-item']",
    "button",
    "a"
]

# Seatmap (container e itens)
SEL_SEATMAP_CONTAINER = "div.seatmap-bus, div[data-testid='seatmap-bus']"
SEL_SEAT_FREE         = "[data-testid='seatmap-item'].seat-free"
SEL_SEAT_OCCUPIED     = "[data-testid='seatmap-item'].seat-occupied"

# Fallbacks globais para fechar (modal/ESC, etc.)
SEL_CLOSE_CANDIDATES_GLOBAL = [
    "[data-testid='close-seatmap']",
    "button[aria-label='Fechar']",
    "button[title='Fechar']",
    "button.close",
    "div[role='dialog'] button",
    "div.seatmap-bus ~ button"
]

# Possíveis abas de deck (1º andar / 2º andar)
DECK_TAB_CANDIDATES = [
    "[role='tab']",
    "[data-testid*='deck']",
    "[data-testid*='andar']",
    "button",
    "a"
]

# =========================
# UTILITÁRIAS
# =========================
def build_driver() -> webdriver.Chrome:
    opts = Options()
    if HEADLESS:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1366,768")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--lang=pt-BR")
    # Evita bloqueios simples
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option('useAutomationExtension', False)

    drv = webdriver.Chrome(options=opts)
    drv.set_page_load_timeout(60)
    return drv

def wait_for_results(driver):
    WebDriverWait(driver, TIMEOUT).until(
        EC.any_of(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, SEL_DEPARTURE_TIME)),
            EC.presence_of_element_located((By.CSS_SELECTOR, SEL_RESULTS_CONTAINER)),
        )
    )

def infinite_scroll(driver, max_scrolls=MAX_SCROLLS, pause=PAUSA_SCROLL):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def safe_text(el) -> str:
    try:
        return el.text.strip()
    except Exception:
        return ""

def open_page(driver, origem_url, destino_url, data_str):
    url = f"https://www.clickbus.com.br/onibus/{origem_url}/{destino_url}?departureDate={data_str}"
    print(f"Acessando: {url}")
    driver.get(url)

def find_trip_cards(driver):
    """Retorna lista (possivelmente vazia) de elementos de card."""
    for css in SEL_TRIP_CARD_CANDIDATES:
        cards = driver.find_elements(By.CSS_SELECTOR, css)
        if cards:
            return cards
    return []

def try_click(el) -> bool:
    try:
        el.click()
        return True
    except ElementClickInterceptedException:
        try:
            el._parent.execute_script("arguments[0].click();", el)
            return True
        except Exception:
            return False
    except Exception:
        return False

def scroll_into_view(driver, el):
    try:
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
        time.sleep(0.25)
    except Exception:
        pass

# =========================
# CARD helpers
# =========================
def _resolve_card_from_line_element(line_el):
    """
    Sobe até o contêiner real do card com base no HTML enviado:
      1) ancestor::div[@data-testid='search-item-container'][1]
      2) ancestor::div[contains(@class,'search-item-main')][1]
    """
    try:
        inner = line_el.find_element(
            By.XPATH,
            "ancestor::div[@data-testid='search-item-container'][1]"
        )
        card = inner.find_element(
            By.XPATH,
            "ancestor::div[contains(@class,'search-item-main')][1]"
        )
        return card
    except Exception:
        # fallback genérico
        curr = line_el
        for _ in range(12):
            try:
                curr = curr.find_element(By.XPATH, "./..")
            except Exception:
                break
            classes = (curr.get_attribute("class") or "").lower()
            dtid    = (curr.get_attribute("data-testid") or "").lower()
            if ("search-item-main" in classes) or (dtid == "search-item-container"):
                return curr
        return line_el

# =========================
# ABRIR / FECHAR MAPA DENTRO DO CARD
# =========================
def _has_visible_seatmap_in_card(card) -> bool:
    try:
        containers = card.find_elements(By.CSS_SELECTOR, SEL_SEATMAP_CONTAINER)
        return any(c.is_displayed() for c in containers)
    except Exception:
        return False

def _wait_seatmap_within_card(card, timeout: int = TIMEOUT) -> bool:
    end = time.time() + timeout
    while time.time() < end:
        if _has_visible_seatmap_in_card(card):
            return True
        time.sleep(0.2)
    return False

def _wait_seatmap_disappear_from_card(card, timeout: int = 10) -> bool:
    end = time.time() + timeout
    while time.time() < end:
        if not _has_visible_seatmap_in_card(card):
            return True
        time.sleep(0.2)
    return False

def abrir_mapa_assentos_no_card(card) -> bool:
    """Abre o seatmap somente dentro do card informado."""
    # Já está aberto?
    try:
        for c in card.find_elements(By.CSS_SELECTOR, SEL_SEATMAP_CONTAINER):
            if c.is_displayed():
                return True
    except Exception:
        pass

    # Botão exato do HTML
    try:
        btns = card.find_elements(By.CSS_SELECTOR, "button[data-testid='select-result-item']")
        for btn in btns:
            scroll_into_view(card._parent, btn)
            if try_click(btn) and _wait_seatmap_within_card(card):
                return True
    except Exception:
        pass

    # Fallback: outros possíveis botões dentro do card
    for css in SEL_SELECT_BUTTONS_OPEN:
        try:
            for btn in card.find_elements(By.CSS_SELECTOR, css):
                scroll_into_view(card._parent, btn)
                if try_click(btn) and _wait_seatmap_within_card(card):
                    return True
        except Exception:
            pass

    # Último recurso: clicar no próprio card
    scroll_into_view(card._parent, card)
    return try_click(card) and _wait_seatmap_within_card(card)

def fechar_mapa_assentos_global(driver) -> bool:
    """Tenta fechar por botões globais, ESC, backdrop."""
    for css in SEL_CLOSE_CANDIDATES_GLOBAL:
        try:
            btns = driver.find_elements(By.CSS_SELECTOR, css)
            for b in btns:
                if b.is_displayed() and try_click(b):
                    time.sleep(0.4)
                    return True
        except Exception:
            pass

    try:
        from selenium.webdriver.common.keys import Keys
        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.ESCAPE)
        time.sleep(0.3)
        return True
    except Exception:
        pass

    try:
        backdrop = driver.find_element(By.CSS_SELECTOR, "[data-testid='modal-backdrop'], .modal-backdrop, .backdrop")
        if backdrop.is_displayed():
            try_click(backdrop)
            time.sleep(0.3)
            return True
    except Exception:
        pass

    return False

def fechar_mapa_assentos_no_card(card) -> bool:
    """Fecha o seatmap do card (toggle ou botão de fechar)."""
    if not _has_visible_seatmap_in_card(card):
        return True

    # 1) tenta o mesmo botão select como toggle
    try:
        for b in card.find_elements(By.CSS_SELECTOR, "button[data-testid='select-result-item']"):
            txt = (b.text or "").strip().lower()
            # Se o site muda o texto para "Fechar", ótimo; senão ainda pode ser toggle
            if "fechar" in txt or True:
                if try_click(b) and _wait_seatmap_disappear_from_card(card):
                    return True
    except Exception:
        pass

    # 2) tenta outros possíveis botões dentro do card
    for sel in SEL_BUTTONS_CLOSE_WITHIN_CARD:
        try:
            for b in card.find_elements(By.CSS_SELECTOR, sel):
                txt = (b.text or "").strip().lower()
                if "fechar" in txt or sel != "button[data-testid='select-result-item']":
                    if try_click(b) and _wait_seatmap_disappear_from_card(card):
                        return True
        except Exception:
            pass

    # 3) fallback global
    if fechar_mapa_assentos_global(card._parent):
        return True

    return not _has_visible_seatmap_in_card(card)

# =========================
# CONTAGEM DE ASSENTOS (DENTRO DO CARD)
# =========================
def _count_seats_in_container(container) -> Dict[str, int]:
    livres  = len(container.find_elements(By.CSS_SELECTOR, SEL_SEAT_FREE))
    ocupados = len(container.find_elements(By.CSS_SELECTOR, SEL_SEAT_OCCUPIED))
    return {"livres": livres, "ocupados": ocupados}

def _iter_decks_and_sum(container) -> Dict[str, int]:
    """Soma 1º/2º andar quando houver tabs; senão conta só o deck atual."""
    total_livres = 0
    total_ocupados = 0

    base = _count_seats_in_container(container)
    total_livres += base["livres"]
    total_ocupados += base["ocupados"]

    try:
        deck_tabs = []
        for css in DECK_TAB_CANDIDATES:
            deck_tabs.extend(container.find_elements(By.CSS_SELECTOR, css))

        tabs_filtradas = []
        seen = set()
        for el in deck_tabs:
            t = (el.text or "").strip().lower()
            if not t:
                continue
            if ("andar" in t) or ("1º" in t) or ("2º" in t) or ("1°" in t) or ("2°" in t) or (t in {"1", "2"}):
                if t not in seen:
                    seen.add(t)
                    tabs_filtradas.append(el)

        for tab in tabs_filtradas:
            if try_click(tab):
                time.sleep(0.6)
                c = _count_seats_in_container(container)
                total_livres += c["livres"]
                total_ocupados += c["ocupados"]
    except Exception:
        pass

    return {"livres": total_livres, "ocupados": total_ocupados}

def contar_assentos_no_card(card, timeout: int = TIMEOUT) -> Dict[str, Optional[int]]:
    """Conta assentos apenas dentro do card atual (somando decks se houver)."""
    try:
        if not _wait_seatmap_within_card(card, timeout):
            return {"assentos_livres": None, "assentos_ocupados": None}

        # pega apenas container que seja DESCENDENTE do card atual
        containers = card.find_elements(By.CSS_SELECTOR, SEL_SEATMAP_CONTAINER)
        container = next((c for c in containers if c.is_displayed()), None)
        if container is None:
            return {"assentos_livres": None, "assentos_ocupados": None}

        somado = _iter_decks_and_sum(container)
        return {"assentos_livres": somado["livres"], "assentos_ocupados": somado["ocupados"]}

    except Exception:
        return {"assentos_livres": None, "assentos_ocupados": None}

# =========================
# COLETA DE UM DIA/ROTA + ASSENTOS
# =========================
def coletar_um_dia(driver, origem_url, destino_url, data_viagem: datetime):
    data_str = data_viagem.strftime("%Y-%m-%d")
    open_page(driver, origem_url, destino_url, data_str)
    wait_for_results(driver)
    time.sleep(2)
    infinite_scroll(driver)

    # Itera pelos CARDS reais
    cards = driver.find_elements(By.CSS_SELECTOR, "div.search-item-main.commodities-enable")
    print(f"→ {data_str} | {origem_url} → {destino_url} | Viagens: {len(cards)}")

    linhas = []
    for i, card in enumerate(cards, start=1):
        # --- extrai valores do próprio card ---
        try:
            empresa_el = card.find_element(By.CSS_SELECTOR, ".company[data-content]")
            empresa = empresa_el.get_attribute("data-content") or safe_text(empresa_el)
        except Exception:
            empresa = ""

        try:
            saida = safe_text(card.find_element(By.CSS_SELECTOR, "time.departure-time"))
        except Exception:
            saida = ""

        try:
            classe = safe_text(card.find_element(By.CSS_SELECTOR, ".service-class"))
        except Exception:
            classe = ""

        try:
            preco_str = safe_text(card.find_element(By.CSS_SELECTOR, ".price-value"))
            preco_str = preco_str.replace("R$", "").replace("\xa0", " ").strip()
            preco = float(preco_str.replace(".", "").replace(",", "."))
        except Exception:
            preco = None

        # --- seatmap estritamente do card ---
        assentos_livres = None
        assentos_ocupados = None
        try:
            scroll_into_view(driver, card)
            if abrir_mapa_assentos_no_card(card):
                contagem = contar_assentos_no_card(card)
                assentos_livres   = contagem.get("assentos_livres")
                assentos_ocupados = contagem.get("assentos_ocupados")

                # fecha ANTES do próximo
                if not fechar_mapa_assentos_no_card(card):
                    fechar_mapa_assentos_global(driver)
                time.sleep(0.2)
            else:
                print(f"[INFO] ({i}) seatmap não abriu; seguindo.")
        except Exception as e:
            print(f"[WARN] ({i}) Falha seatmap: {e}")

        linhas.append({
            "data_viagem": data_str,
            "empresa": empresa,
            "horario_saida": saida,
            "classe": classe,
            "preco": preco,
            "assentos_livres": assentos_livres,
            "assentos_ocupados": assentos_ocupados,
        })

    return linhas

# =========================
# COLETA POR TRECHO (RETORNA UM ÚNICO DF)
# =========================
def coletar_por_trecho(rotas: List[Dict[str, str]], dias: int = DIAS_A_COLETAR) -> pd.DataFrame:
    """Retorna um único DataFrame com todas as rotas + coluna 'trecho'."""
    if dias < 1:
        dias = 1
    hoje = datetime.now().date()

    todas_linhas = []

    for idx, rota in enumerate(rotas, start=1):
        origem_url  = rota["origem_url"]
        destino_url = rota["destino_url"]
        nome_trecho = f"{origem_url} → {destino_url}"

        print("=" * 80)
        print(f"[{idx}/{len(rotas)}] Trecho: {origem_url} -> {destino_url}")

        driver = build_driver()
        try:
            for i in range(dias):
                dia = hoje + timedelta(days=i)
                print("-" * 80)
                print(f"Dia {i+1}/{dias} | {dia}")
                try:
                    linhas = coletar_um_dia(driver, origem_url, destino_url, dia)
                    for l in linhas:
                        l["trecho"] = nome_trecho
                    todas_linhas.extend(linhas)
                except Exception as e:
                    print(f"[ERRO] {dia} {origem_url}->{destino_url}: {e}")
                    try: driver.quit()
                    except: pass
                    driver = build_driver()
                time.sleep(1.5)
        finally:
            try: driver.quit()
            except: pass

    df_final = pd.DataFrame(todas_linhas)
    if not df_final.empty:
        df_final["data_viagem"] = pd.to_datetime(df_final["data_viagem"], errors="coerce")
        df_final = (
            df_final
            .drop_duplicates()
            .sort_values(["trecho", "data_viagem", "horario_saida"], na_position="last")
            .reset_index(drop=True)
        )
    return df_final

# =========================
# EXECUÇÃO
# =========================
if __name__ == "__main__":
    df_resultado = coletar_por_trecho(ROTAS, dias=DIAS_A_COLETAR)

    if PRINT_ALL_ROWS:
        pd.set_option("display.max_rows", None)
        pd.set_option("display.max_columns", None)
        pd.set_option("display.width", None)         # autoajuste ao terminal
        pd.set_option("display.max_colwidth", None)  # não truncar texto

    # Preview e salvamento por trecho, SEM a coluna 'trecho'
    if not df_resultado.empty:
        for trecho, dfg in df_resultado.groupby("trecho", sort=False):
            dfg = dfg.drop(columns=["trecho"])
            print("\n" + "=" * 80)
            print(f"===== TRECHO: {trecho} =====")
            print(dfg.reset_index(drop=True))

[1/2] Trecho: vitoria-da-conquista-ba-todos -> sao-paulo-sp-todos
--------------------------------------------------------------------------------
Dia 1/1 | 2025-08-23
Acessando: https://www.clickbus.com.br/onibus/vitoria-da-conquista-ba-todos/sao-paulo-sp-todos?departureDate=2025-08-23
→ 2025-08-23 | vitoria-da-conquista-ba-todos → sao-paulo-sp-todos | Viagens: 26
[2/2] Trecho: sao-paulo-sp-todos -> vitoria-da-conquista-ba-todos
--------------------------------------------------------------------------------
Dia 1/1 | 2025-08-23
Acessando: https://www.clickbus.com.br/onibus/sao-paulo-sp-todos/vitoria-da-conquista-ba-todos?departureDate=2025-08-23
→ 2025-08-23 | sao-paulo-sp-todos → vitoria-da-conquista-ba-todos | Viagens: 18

===== TRECHO: sao-paulo-sp-todos → vitoria-da-conquista-ba-todos =====
   data_viagem       empresa horario_saida               classe   preco  \
0   2025-08-23       Gontijo         10:30            Executivo  295.66   
1   2025-08-23       Gontijo         10:30

In [ ]:
# Inclui data e hora no nome do arquivo para evitar sobrescrever
datahora_str1 = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
nome_arquivo = f"passagens_clickbus_rotas_{datahora_str}.xlsx"# Exporta todos os dataframes para um único arquivo Excel

with pd.ExcelWriter(nome_arquivo, engine="openpyxl") as writer:
    for nome_trecho, df in dfs.items():
        sheet_name = nome_trecho[:31]
        df.to_excel(writer, sheet_name=sheet_name, index=False)